# **사전 준비**

In [ ]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu

In [ ]:
from google.colab import drive
import os

# 먼저 구글 드라이브 마운트
drive.mount('/content/drive')

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

# 환경 변수에서 API 키 가져오기
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = (
    "/content/drive/MyDrive/langchain-tutorial/Ch04. Advanced Rag/Data/투자설명서.pdf"
)
loader = PyPDFLoader(file_path)

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap = 100)

docs = loader.load_and_split(doc_splitter)

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

# 데이터를 임베딩으로 변환
embedding = OpenAIEmbeddings(model="text-embeddings-3-large")

In [ ]:
# FAISS 라이브러리 임포트
from langchain_community.vectorstores import FAISS

# FAISS 벡터 스토어 생성
faiss_store = FAISS.from_documents(docs, embedding)

# FAISS 벡터 스토어 저장
persist_directory = "/content/DB"
faiss_store.save_local(persist_directory)

In [ ]:
# 저장한 FAISS DB 불러오기
vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserializtion=True)